In [1]:
import rdflib
from rdflib import Graph, Namespace
import owlrl
from rdflib.namespace import FOAF, DCTERMS, SKOS

In [2]:
dbo = Namespace("http://dbpedia.org/ontology/")

In [3]:
my_rdf = Graph().parse('data.ttl', format='turtle' )
print(my_rdf.serialize(format="turtle"))

@prefix city: <http://example.org/city/> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix loria: <http://exemple.org/ontology/loria/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

city:Paris loria:nearTo city:Versailles .

loria:nearTo a owl:SymmetricProperty ;
    rdfs:domain dbo:PopulatedPlace ;
    rdfs:range dbo:PopulatedPlace .




In [4]:
owlrl.DeductiveClosure(owlrl.RDFS_Semantics, datatype_axioms=True).expand(my_rdf)
owlrl.DeductiveClosure(owlrl.OWLRL_Extension, datatype_axioms=True).expand(my_rdf)

In [5]:
spql = 'SELECT ?x WHERE { ct:Versailles loria:nearTo ?x .}'
res = my_rdf.query(spql, initNs={"loria":"http://exemple.org/ontology/loria/", "ct":"http://example.org/city/" })

for row in res:
  print(row["x"])


http://example.org/city/Paris


In [6]:
spql = 'select ?x where { ct:Versailles a ?x . }'
res = my_rdf.query(spql, initNs={"dbo":dbo, "ct":"http://example.org/city/"})

for row in res:
  print(row)

(rdflib.term.URIRef('http://dbpedia.org/ontology/PopulatedPlace'),)
(rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Resource'),)
(rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Thing'),)


### Select Well-known Ontologies

In [7]:
dc_onto = Graph().parse("https://www.dublincore.org/specifications/dublin-core/dcmi-terms/dublin_core_terms.ttl", format="turtle")
foaf_onto = Graph().parse("http://xmlns.com/foaf/0.1/index.rdf", format="xml")
skos_onto = Graph().parse("http://www.w3.org/TR/skos-reference/skos.rdf", format="xml")
pp = Namespace("http://example.org/people/")

### Create your Customed Ontology

In [8]:
rc_onto = Graph().parse('loria_onto.ttl', format='turtle' )
print("LORIA ONTO --> ....",rc_onto.serialize(format="turtle"))

LORIA ONTO --> .... @prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix loria: <http://exemple.org/ontology/loria/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

ex:Fantacy skos:broaderTransitive ex:Story .

loria:write rdfs:domain foaf:Person ;
    rdfs:range foaf:Document ;
    rdfs:subPropertyOf dcterms:creator ;
    owl:inverseOf loria:writtenBy .




### Create your RDF Data

In [9]:
my_abox = Graph().parse('my_rdf.ttl', format='turtle' )
print(my_abox.serialize(format="turtle"))

@prefix ex: <http://example.org/> .
@prefix loria: <http://exemple.org/ontology/loria/> .
@prefix pp: <http://example.org/people/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

pp:jkrowling loria:write ex:harry_potter .

ex:harry_potter skos:broaderTransitive ex:Fantacy .




### Combine data and ontologies

In [10]:
g = dc_onto + foaf_onto + skos_onto + rc_onto + my_abox

### Deductive Closure

In [11]:
owlrl.DeductiveClosure(owlrl.RDFS_Semantics, datatype_axioms=True).expand(g)
owlrl.DeductiveClosure(owlrl.OWLRL_Extension, datatype_axioms=True).expand(g)

### Query

In [12]:
# What is Harry Potter?

spql = 'SELECT ?p ?o WHERE { ex:harry_potter ?p ?o.}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/"})

for row in res:
  print(row["p"], row["o"])

http://www.w3.org/2004/02/skos/core#broaderTransitive http://example.org/Fantacy
http://www.w3.org/2004/02/skos/core#broaderTransitive http://example.org/Story
http://www.w3.org/2004/02/skos/core#semanticRelation http://example.org/Fantacy
http://www.w3.org/2004/02/skos/core#semanticRelation http://example.org/Story
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Document
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/wordnet/1.6/Document
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2004/02/skos/core#Concept
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing
http://www.w3.org/1999/02/22-rdf-syntax-ns#type N6c393f5b320241da87e000e692c7d941
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/wordnet/1.6/Agent-3
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Ag

In [13]:
# What is JK Rowling?

spql = 'SELECT ?p ?o WHERE { pp:jkrowling ?p ?o.}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/"})

for row in res:
  print(row["p"], row["o"])

http://exemple.org/ontology/loria/write http://example.org/harry_potter
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/10/swap/pim/contact#Person
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/wordnet/1.6/Person
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Agent
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/wordnet/1.6/Agent-3
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing
http://purl.org/dc/terms/creator http://example.org/harry_potter
http://purl.org/dc/terms/contributor http://example.org/harry_potter
http://purl.org/dc/elements/1.1/contributor http://example.org/harry_potter
http://purl

## To Query with well-known Terms

In [14]:
# Who are Person?

spql = 'SELECT ?x WHERE { ?x a foaf:Person .}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/"})

for row in res:
  print(row["x"])

http://example.org/people/jkrowling


In [15]:
# What are Documents?

spql = 'SELECT ?x WHERE { ?x a foaf:Document .}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/"})

for row in res:
  print(row["x"])

http://example.org/harry_potter


In [16]:
# Who write what?

spql = 'SELECT ?x ?y WHERE { ?x dcterms:creator ?y .}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/", "dcterms":DCTERMS})

for row in res:
  print(row["x"], row["y"])

http://www.w3.org/2004/02/skos/core Sean Bechhofer
http://www.w3.org/2004/02/skos/core Alistair Miles
http://example.org/people/jkrowling http://example.org/harry_potter


In [17]:
# What are boader Concepts of harry potter

spql = 'SELECT ?x WHERE { ex:harry_potter skos:broaderTransitive ?x .}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/", "dcterms":DCTERMS, "skos":SKOS})

for row in res:
  print(row["x"])

http://example.org/Fantacy
http://example.org/Story


In [18]:
# What are boader Concepts of Fantacy

spql = 'SELECT ?x WHERE { ex:Fantacy skos:broaderTransitive ?x .}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/", "dcterms":DCTERMS, "skos":SKOS})

for row in res:
  print(row["x"])

http://example.org/Story


In [19]:
# What are narrower Concepts of Fantacy

spql = 'SELECT ?x WHERE { ex:Fantacy skos:narrowerTransitive ?x .}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/", "dcterms":DCTERMS, "skos":SKOS})

for row in res:
  print(row["x"])

http://example.org/harry_potter


In [20]:
# What are narrower Concepts of Story

spql = 'SELECT ?x WHERE { ex:Story skos:narrowerTransitive ?x .}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/", "dcterms":DCTERMS, "skos":SKOS})

for row in res:
  print(row["x"])

http://example.org/Fantacy
http://example.org/harry_potter


In [21]:
# ASK

spql = 'ASK {ex:harry_potter skos:broaderTransitive ex:Story .}'
res = g.query(spql, initNs={"foaf":FOAF, "pp":pp, "ex":"http://example.org/", "dcterms":DCTERMS, "skos":SKOS})

for row in res:
  print(row)

True


In [22]:
texte = input("Entrez une chaîne de caractères : ")
print("Vous avez entré : " + texte)

Vous avez entré : 
